![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Let's go!

In [1]:
%pip install selenium
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from facebook_scraper import get_posts, get_profile
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime
from ast import literal_eval
import numpy as np
import csv
import os
import re

Note: you may need to restart the kernel to use updated packages.


Account, Facebook login password and FanPage ID we need to Crawl

In [2]:
EMAIL = "pduc35395@gmail.com"
PASSWORD = "a_E.rn2B6D7tz2S"
PAGE_ID = "HarryMaguire93"

In [3]:
def initDriver():
    """
    Initialize the Chrome driver with options to disable notifications.
    This effectively creates a new browser window.
    """
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-notifications")
    return webdriver.Chrome(options=options)

def fbLogin(driver, email, password):
    """
    Log in to Facebook using the provided email and password after 
    the browser window is opened.
    """
    driver.get("https://facebook.com")

    # Login process
    userEmail = driver.find_element(by="id", value="email")
    userEmail.send_keys(email)
    userPass = driver.find_element(by="id", value="pass")
    userPass.send_keys(password)
    userPass.send_keys(Keys.ENTER)

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Let's continue to scratch all missing reactions and comments with selenium

In [4]:
# This was written few days later
# I found out that the facebook_scraper library is not stable
# So let's load up our data and strip data we won't need, fill in the missing data with different method
df = pd.read_csv('Data/raw_Harry93.csv')
df.head(5)

,original_request_url,post_url,post_id,text,post_text,shared_text,original_text,time,timestamp,image,...,with,page_id,sharers,image_id,image_ids,video_ids,videos,was_live,fetched_time,header
0,https://facebook.com/pfbid037nJvE8Y3buLNNRezqZ...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid037nJvE8Y3buLNNRezqZS6L6mHoByD8NxdLUwXAp8...,Thanks to our away fans for the unbelievable s...,Thanks to our away fans for the unbelievable s...,NaN,NaN,2023-11-22 02:14:00,NaN,https://scontent.fhan14-1.fna.fbcdn.net/v/t39....,...,NaN,NaN,NaN,NaN,[],[],[],False,2023-12-03 09:33:00.841303,NaN
1,https://facebook.com/pfbid02YkYeWAcdyv2XkfBP3Z...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02YkYeWAcdyv2XkfBP3ZYjEtLo2DpuEu2WxbG1B89...,The time to tackle online abuse in football is...,The time to tackle online abuse in football is...,STRIVER.CO.UK\nStriver – The Alternative to So...,NaN,2023-11-14 19:07:00,NaN,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,False,2023-12-03 09:33:36.477919,NaN
2,https://facebook.com/pfbid023a7jkBr9CWXmTwX27j...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid023a7jkBr9CWXmTwX27jkU4q7avRWicnMbfdnuZey...,A big 3 points in London 💪❤️,A big 3 points in London 💪❤️,NaN,NaN,2023-11-05 02:14:00,NaN,https://scontent.fhan14-4.fna.fbcdn.net/v/t39....,...,NaN,NaN,NaN,NaN,[],[],[],False,2023-12-03 09:34:34.054792,NaN
3,https://facebook.com/pfbid02TUYFtHr8h17Qipuh5A...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02TUYFtHr8h17Qipuh5AvBwDF3mB18u1zAx1vUr7Q...,A win dedicated to Sir Bobby Charlton. A legen...,A win dedicated to Sir Bobby Charlton. A legen...,NaN,NaN,2023-10-22 04:49:00,NaN,https://scontent.fhan14-4.fna.fbcdn.net/v/t39....,...,NaN,NaN,NaN,NaN,[],[],[],False,2023-12-03 09:36:03.251149,NaN
4,https://facebook.com/pfbid02yZptV8GSxyJ6W3iRXs...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02yZptV8GSxyJ6W3iRXs81QfxMpQVyYjPiAzmi38F...,Special night at Wembley - Qualification secur...,Special night at Wembley - Qualification secur...,NaN,NaN,2023-10-18 14:24:00,NaN,https://scontent.fhan14-1.fna.fbcdn.net/v/t39....,...,NaN,NaN,NaN,NaN,[],[],[],False,2023-12-03 09:37:00.501391,NaN


In [5]:
# Drop unnecessary columns
df = df[['post_id','text', 'time', 'comments', 'shares']]
# TODO:
# Add reactions count for each post
# Add comments text for each post
df.head(5)

,post_id,text,time,comments,shares
0,pfbid037nJvE8Y3buLNNRezqZS6L6mHoByD8NxdLUwXAp8...,Thanks to our away fans for the unbelievable s...,2023-11-22 02:14:00,473.0,24
1,pfbid02YkYeWAcdyv2XkfBP3ZYjEtLo2DpuEu2WxbG1B89...,The time to tackle online abuse in football is...,2023-11-14 19:07:00,92.0,5
2,pfbid023a7jkBr9CWXmTwX27jkU4q7avRWicnMbfdnuZey...,A big 3 points in London 💪❤️,2023-11-05 02:14:00,2443.0,199
3,pfbid02TUYFtHr8h17Qipuh5AvBwDF3mB18u1zAx1vUr7Q...,A win dedicated to Sir Bobby Charlton. A legen...,2023-10-22 04:49:00,913.0,108
4,pfbid02yZptV8GSxyJ6W3iRXs81QfxMpQVyYjPiAzmi38F...,Special night at Wembley - Qualification secur...,2023-10-18 14:24:00,1548.0,133


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Function Crawls all reactions

In [6]:
def get_count_react(href):
        ''' 
        Get reaction type and count from the given href
        '''
        reaction_type = re.search(r'reaction_type=(\d+)', href).group(1)
        total_count = re.search(r'total_count=(\d+)', href).group(1)
        if reaction_type == '1':
            reaction_type = 'like'
        elif reaction_type == '2':
                reaction_type = 'love'
        elif reaction_type == '16':
               reaction_type = 'care'
        elif reaction_type == '4':
               reaction_type = 'haha'
        elif reaction_type == '3':
                reaction_type = 'wow'
        elif reaction_type == '7':
                reaction_type = 'sad'
        else:
                reaction_type = 'angry'
        return reaction_type, int(total_count)

In [7]:
def get_reactions(driver, post_id):
    '''
    Get all reactions of a post
    '''
    driver.get(f"https://mbasic.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier={post_id}")
    sleep(10)
    links = driver.find_elements(By.XPATH, value="//a[@role='button' and @aria-pressed='false']")
    
    hrefs = [link.get_attribute('href') for link in links]
    reactions = {}
    for href in hrefs:
        reaction_type, count = get_count_react(href)
        reactions[reaction_type] = count
    return reactions

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Funtion Crawls comment_full

In [8]:
def get_comments_per_site(driver):
    '''
    Get all comments appearing on site max 10
    {
        'commenter_name': name,
        'commenter_id': commenter_id,
        'text': text,
        'comment_id': comment_id,
    }
    '''
    comments = []
    
    boxes = driver.find_elements(By.XPATH, value="//html/body/div/div/div[2]/div/div/div[2]/div/div[5]/*")
    
    for box in boxes:
        try:
            commenter_name = box.find_element(By.XPATH, value=".//h3/a").text
            def get_id_from_href(href):
                id_match = re.search(r'\?id=(\d+)', href)
                if id_match:
                    return id_match.group(1)
                else:
                    return re.search(r'/([^?]+)', href).group(1)
            commenter_id =  get_id_from_href(box.find_element(By.XPATH, value=".//h3/a").get_attribute('href'))
            comment_text = box.find_element(By.XPATH, value=".//div/div").text
            comment_id = box.get_attribute('id')
            comments.append({
                'commenter_name': commenter_name,
                'commenter_id': commenter_id,
                'text': comment_text,
                'comment_id': comment_id,
            })
        except: # Have see previous
            continue

    return comments

In [9]:
def get_comment_full(driver, amount, post_id):
    ''' 
    Get <amount> comments of a post
    '''
    driver.get(f"https://mbasic.facebook.com/{post_id}")
    sleep(10)

    comments = []
    while True:
        comments += get_comments_per_site(driver)
        if len(comments) >= amount:
            break
        sleep(20)
    
        # Find see_next else break
        try:
            button = driver.find_element(By.XPATH, value="//div[contains(@id, 'see_next')]")
            button.find_element(By.XPATH, value=".//a").click()
            sleep(10)
        except:
            break
    return comments[:amount]

In [10]:
driver = initDriver()
fbLogin(driver, EMAIL, PASSWORD)
sleep(5)

In [11]:
df['reactions'] = df['post_id'].apply(lambda x: get_reactions(driver, x))
df['reaction_count'] = df['reactions'].apply(lambda x: sum(x.values()))
df['comments_full'] = df['post_id'].apply(lambda x: get_comment_full(driver, 20, x)) # Get 20 comments for each post

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Convert_to_datetime

In [1]:
df['time'] = df['time'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M"))
print(df.head())

                                             post_id  \
0  pfbid037nJvE8Y3buLNNRezqZS6L6mHoByD8NxdLUwXAp8...   
1  pfbid02YkYeWAcdyv2XkfBP3ZYjEtLo2DpuEu2WxbG1B89...   
2  pfbid023a7jkBr9CWXmTwX27jkU4q7avRWicnMbfdnuZey...   
3  pfbid02TUYFtHr8h17Qipuh5AvBwDF3mB18u1zAx1vUr7Q...   
4  pfbid02yZptV8GSxyJ6W3iRXs81QfxMpQVyYjPiAzmi38F...   

                                                text                time  \
0  Thanks to our away fans for the unbelievable s... 2023-11-22 02:14:00   
1  The time to tackle online abuse in football is... 2023-11-14 19:07:00   
2                       A big 3 points in London 💪❤️ 2023-11-05 02:14:00   
3  A win dedicated to Sir Bobby Charlton. A legen... 2023-10-22 04:49:00   
4  Special night at Wembley - Qualification secur... 2023-10-18 14:24:00   

   comments  shares                                          reactions  \
0       473      24  {'like': 13176, 'love': 2421, 'care': 201, 'ha...   
1        92       5  {'like': 1017, 'love': 170, 'haha': 2

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Export to final file

In [2]:
df.to_csv('Data/Final_Harry93.csv', index=False)